# Get Restaurants in different neighborhoods in Toronto

We are going to obtain restaurants name and geo-coordinates that are present in different neighborhoods of Toronto. To achieve this we get this data from Foursquare API. (https://foursquare.com/)

1. Read in the dataframe that contains neighborhoods and there geo-coordinates.
2. Based on the geo-coordinates of the neighborhood, get the restaurant data for each neighborhood.
3. Read the restaurant and their geo-coordinate data along with the neighborhoods into a dataframe.

Please use this link to open the Jupyter notebook to see the maps:

https://nbviewer.jupyter.org/

Import neccessary libraries.

In [1]:
import numpy as np
import pandas as pd
import requests
import folium

**Read the neighborhood dataframe from data collected earlier.**

In [2]:
Toronto_data=pd.read_csv('Neighborhoods_Toronto.csv')

In [3]:
Toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:
print('Number of rows',Toronto_data.shape[0])

Number of rows 103


## Getting data from foursquare API

Setting the credentials for foursquare API app that was created earlier using foursquare developer account at https://foursquare.com/developers/apps

In [5]:
# content removed for privacy
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: J1HNQG2BTTUH5JXPZLFL0NRNEGY5UFE3JIZCBJO3XESMYNWQ
CLIENT_SECRET:1LIFVE0DGY2M431JARDDM4WHABK1FP2O0LWDQW1HQFZL11FF


**Let's create a function to access the foursquare API and search for restaurants in neighborhoods of Toronto based on the geo-coordinates.**

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    food_venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query=Restaurant'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng)
            
        # make the GET request
        results = requests.get(url).json()['response']['venues']#[0]['items']
# #         # return only relevant information for each nearby venue
        food_venues_list.append([(
              name, 
              lat, 
              lng, 
              v['name'], 
              v['location']['lat'], 
              v['location']['lng']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in food_venues_list for item in venue_list])
    return (nearby_venues)

Let's run the above function.

In [7]:
Toronto_food_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Malvern, Rouge
Highland Creek, Port Union, Rouge Hill
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South, Flemingdon Park
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
Riverdale, The Danf

Finally, give the restaurant dataframe columns name.

In [8]:
Toronto_food_venues.columns=['Neighborhood','Neighborhood_lat','Neighborhood_long',
                             'Restaurant','Restaurant_lat','Restaurant_long']

## The Final restaurant and neighborhood dataframe.

In [9]:
Toronto_food_venues.head(10)

,Neighborhood,Neighborhood_lat,Neighborhood_long,Restaurant,Restaurant_lat,Restaurant_long
0,"Malvern, Rouge",43.806686,-79.194353,Ted's Restaurant,43.784468,-79.169200
1,"Malvern, Rouge",43.806686,-79.194353,Perfect Chinese Restaurant 雅瓊海鮮酒家,43.787774,-79.270294
2,"Malvern, Rouge",43.806686,-79.194353,Alton Restaurant 益街坊,43.825582,-79.276038
3,"Malvern, Rouge",43.806686,-79.194353,東海漁村 Tasty BBQ Seafood Restaurant (Tasty BBQ S...,43.794425,-79.353300
4,"Malvern, Rouge",43.806686,-79.194353,Federick Restaurant,43.851124,-79.253210
5,"Malvern, Rouge",43.806686,-79.194353,Inware Restaurant Equipment and Supplies 宏鑫餐具,43.804130,-79.296060
6,"Malvern, Rouge",43.806686,-79.194353,Perkins Restaurant & Bakery,43.829004,-79.097694
7,"Malvern, Rouge",43.806686,-79.194353,Ming's Restaurant 大明茶餐廳,43.861347,-79.304258
8,"Malvern, Rouge",43.806686,-79.194353,Fortune House Restaurant 富記,43.790025,-79.302509
9,"Malvern, Rouge",43.806686,-79.194353,The Royal Chinese Restaurant 避風塘小炒,43.780505,-79.298844


In [10]:
print('Number of rows',Toronto_food_venues.shape[0])

Number of rows 3090


## Saving the data to csv file

In [11]:
Toronto_food_venues.to_csv('Toronto_Restaurants.csv',index=False)